In [ ]:
from selenium import webdriver
from selenium. webdriver. chrome. service import Service
from selenium. webdriver. chrome. options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from tqdm import tqdm
import pickle

In [ ]:
# 65637item 456ページ
# 3時間弱でスクレイピング可能見込み
urls = [f"https://www.partzilla.com/search?ui=typeahead&filters=%5B%7B%22name%22%3A%22manufacturer%22%2C%22values%22%3A%5B%22SUZUKI%22%5D%7D%2C%7B%22name%22%3A%22sort%22%2C%22values%22%3A%5B%22score+desc%22%5D%7D%5D&pg={i}&pageSize=144" for i in range(1, 457)]

In [ ]:
all_items = []
for url in tqdm(urls):
    options = Options ()
    options.binary_location = "/usr/bin/chromium-browser" # Chromium のパス
    # options.add_argument ("--headless")
    options.add_argument ("--no-sandbox")
    options.add_argument ("--disable-dev-shm-usage")
    service = Service ("/usr/bin/chromedriver") # chromedriverのパス
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    # ページの読み込み完了を待機
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-card")))    
    cards = driver.find_elements(By.CLASS_NAME, "product-card")
    items = []
    for card in cards:
        item = {}
        name = card.find_element(By.CLASS_NAME, "product-name").text
        type_number =card.find_element(By.CSS_SELECTOR, '[style="line-height: 1;"]').text.replace("\n", " ")
        price = float(card.find_element(By.CLASS_NAME, "sell-price").text.replace("$", "").replace(",", ""))
        item["name"] = name
        item["type_number"] = type_number
        item["price"] = price
        items.append(item) 
    all_items.extend(items)
    driver.quit()

100%|██████████| 3/3 [00:45<00:00, 15.09s/it]


In [6]:
len(all_items)

432

In [ ]:
result_df = pd.DataFrame(all_items)
result_df

In [ ]:
result_df.to_csv("suzuki_parts.csv", index=False)